In [ ]:
'''Download the files '''
'''Only for colab'''

!wget https://github.com/lacykaltgr/continual-learning-ait/archive/refs/heads/experiment.zip
!unzip experiment.zip
!find continual-learning-ait-experiment -type f ! -name "main.ipynb" -exec cp {} . \;

!rm -r stable_diffusion
!rm -r models
!mkdir stable_diffusion
!mkdir models
!mv diffusion_model.py stable_diffusion/
!mv autoencoder_kl.py stable_diffusion/
!mv layers.py stable_diffusion/
!mv stable_diffusion.py stable_diffusion/
!mv constants.py stable_diffusion/
!mv encoder.h5 models/
!mv classifier.h5 models/

In [1]:
from keras.models import load_model
import tensorflow as tf
import keras
import numpy as np
from stable_diffusion.layers import PaddedConv2D, apply_seq
from stable_diffusion.diffusion_model import ResBlock, Downsample, Upsample
import math
from stable_diffusion.constants import _ALPHAS_CUMPROD, PYTORCH_CKPT_MAPPING

RuntimeError: module compiled against API version 0x10 but this version of numpy is 0xf

RuntimeError: module compiled against API version 0x10 but this version of numpy is 0xf

ImportError: numpy.core._multiarray_umath failed to import

ImportError: numpy.core.umath failed to import

RuntimeError: module compiled against API version 0x10 but this version of numpy is 0xf

...

ImportError: numpy.core.umath failed to import

RuntimeError: module compiled against API version 0x10 but this version of numpy is 0xf

ImportError: numpy.core._multiarray_umath failed to import

ImportError: numpy.core.umath failed to import

TypeError: Unable to convert function return value to a Python type! The signature was
	() -> handle

In [ ]:
def load_cifar_10():
    (X_train, y_train), (X_test, y_test) = tf.keras.datasets.cifar10.load_data()
    n_classes = 10
    X_train = (X_train / 127.5) -1
    X_test = (X_test / 127.5) -1
    y_train = tf.keras.utils.to_categorical(y_train, n_classes)
    y_test = tf.keras.utils.to_categorical(y_test, n_classes)
    return (X_train, y_train), (X_test, y_test)

In [ ]:
encoder = load_model("models/encoder.h5")
classifier = load_model("models/classifier.h5")

In [ ]:
class UNetModel(keras.Model):
    def __init__(self):
        print("UNetModel init")
        super().__init__()
        self.time_embed = [
            keras.layers.Dense(1280),
            keras.activations.swish,
            keras.layers.Dense(1280),
        ]
        self.input_blocks = [
            [PaddedConv2D(320, kernel_size=3, padding=1)],
            [ResBlock(320, 320)],
            [ResBlock(320, 320)],
            [Downsample(320)],
            [ResBlock(320, 640)], #, Conv2D(640, kernel_size=8, padding="same")],
            [ResBlock(640, 640)], #, Conv2D(640, kernel_size=8, padding="same")],
            #[Downsample(640)],
            #[ResBlock(640, 1280)], #, Conv2D(1280, kernel_size=8, padding="same")],
            #[ResBlock(1280, 1280)], #, Conv2D(1280, kernel_size=8, padding="same")],
            #[Downsample(1280)],
            #[ResBlock(1280, 1280)],
            #[ResBlock(1280, 1280)],
        ]
        self.middle_block = [
            ResBlock(640, 640),
            #Conv2D(1280, kernel_size=8, padding="same"),
            ResBlock(640, 640),
        ]
        self.output_blocks = [
            #[ResBlock(2560, 1280)],
            #[ResBlock(2560, 1280)],
            #[ResBlock(2560, 1280), Upsample(1280)],
            #[ResBlock(2560, 1280)], #, Conv2D(1280, kernel_size=8, padding="same")],
            #[ResBlock(2560, 1280)], #, Conv2D(1280, kernel_size=8, padding="same")],
            #[
            #    ResBlock(1920, 1280),
            #    #Conv2D(1280, kernel_size=8, padding="same"),
            #    Upsample(1280),
            #],
            [ResBlock(1920, 640)], #, Conv2D(640, kernel_size=8, padding="same")],  # 6
            [ResBlock(1280, 640)], #, Conv2D(640, kernel_size=8, padding="same")],
            [
                ResBlock(960, 640),
                #Conv2D(640, kernel_size=8, padding="same"),
                Upsample(640),
            ],
            [ResBlock(960, 320)], #, Conv2D(320, kernel_size=8, padding="same")],
            [ResBlock(640, 320)], #, Conv2D(320, kernel_size=8, padding="same")],
            [ResBlock(640, 320)], #, Conv2D(320, kernel_size=8, padding="same")],
        ]
        self.out = [
            keras.layers.GroupNormalization(epsilon=1e-5),
            keras.activations.swish,
            PaddedConv2D(4, kernel_size=3, padding=1),
        ]

    def call(self, inputs):
        x, t_emb = inputs
        emb = apply_seq(t_emb, self.time_embed)

        def apply(x, layer):
            return layer([x, emb]) if isinstance(layer, ResBlock) else layer(x)

        saved_inputs = []
        for b in self.input_blocks:
            for layer in b:
                x = apply(x, layer)
            saved_inputs.append(x)

        for layer in self.middle_block:
            x = apply(x, layer)

        for b in self.output_blocks:
            x = tf.concat([x, saved_inputs.pop()], axis=-1)
            for layer in b:
                x = apply(x, layer)

        return apply_seq(x, self.out)

    def initialize(self, params, input_latent=None, batch_size=64):
        timesteps = np.arange(1, params['num_steps']+ 1)
        input_lat_noise_t = timesteps[int(len(timesteps)* params["input_latent_strength"])]
        latent, alphas, alphas_prev = self.get_starting_parameters(
            timesteps, batch_size, input_latent=input_latent, input_lat_noise_t=input_lat_noise_t
        )
        timesteps = timesteps[: int(len(timesteps)*params["input_latent_strength"])]
        return latent, alphas, alphas_prev, timesteps


    def get_x_prev(self, x, e_t, a_t, a_prev, temperature):
        sigma_t = 0
        sqrt_one_minus_at = math.sqrt(1 - a_t)
        pred_x0 = x - sqrt_one_minus_at * e_t / math.sqrt(a_t)

        # Direction pointing to x_t
        dir_xt = math.sqrt(1.0 - a_prev - sigma_t**2) * e_t
        #noise = sigma_t * tf.random.normal(x.shape, seed=seed) * temperature
        x_prev = math.sqrt(a_prev) * pred_x0 + dir_xt
        return x_prev


    def get_model_output(self, latent, timestep, batch_size):
        timesteps = tf.convert_to_tensor([timestep], dtype=tf.float32)
        t_emb = self.timestep_embedding(timesteps)
        t_emb = tf.repeat(t_emb, repeats=batch_size, axis=0)
        latent = self.call([latent, t_emb])
        return latent


    def timestep_embedding(self, timesteps, dim=320, max_period=10000):
        half = dim // 2
        freqs = np.exp(
            -math.log(max_period) * np.arange(0, half, dtype="float32") / half
        )
        args = np.array(timesteps) * freqs
        embedding = np.concatenate([np.cos(args), np.sin(args)])
        return tf.convert_to_tensor(embedding.reshape(1, -1), dtype=self.dtype)



    # for model with input latent

    def add_noise(self, x, t, noise=None):
        if len(x.shape) == 3:
            x = tf.expand_dims(x, axis=0)
        batch_size, w, h, c = x.shape[0], x.shape[1], x.shape[2], x.shape[3]
        if noise is None:
            noise = tf.random.normal((batch_size, w, h, c), dtype=tf.float32)
        sqrt_alpha_prod = tf.cast(_ALPHAS_CUMPROD[t] ** 0.5, tf.float32)
        sqrt_one_minus_alpha_prod = (1 - _ALPHAS_CUMPROD[t]) ** 0.5

        return sqrt_alpha_prod * x + sqrt_one_minus_alpha_prod * noise

    def get_starting_parameters(self, timesteps, batch_size,  input_latent=None, input_lat_noise_t=None):
        n_h = self.img_height // 8
        n_w = self.img_width // 8
        alphas = [_ALPHAS_CUMPROD[t] for t in timesteps]
        alphas_prev = [1.0] + alphas[:-1]
        if input_latent is None:
            latent = tf.random.normal((batch_size, n_h, n_w, 4))
        else:
            input_latent = tf.cast(input_latent, self.dtype)
            #latent = tf.repeat(input_latent , batch_size , axis=0)
            latent = self.add_noise(input_latent, input_lat_noise_t)
        return latent, alphas, alphas_prev


In [ ]:
'''Generate samples and train the diffusion model at the same time'''

def generate(cls=classifier, input_latent=None, train=True, coeff=1.0):

    batch_size = params['batch_size'] if train else 64
    latent, alphas, alphas_prev, timesteps = model.initialize(params, input_latent, batch_size)


    for index, timestep in reversed(list(enumerate(timesteps))):
        if train:
            with tf.GradientTape() as tape:
                e_t = model.get_model_output(
                    latent,
                    timestep,
                    batch_size,
                )
                a_t, a_prev = alphas[index], alphas_prev[index]
                latent = model.gen.get_x_prev(latent, e_t,  a_t, a_prev, params["temperature"])

                pred = cls(latent)
                #loss based on confidence
                #ENT = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y_pre, logits=y_pre))
                loss = coeff*tf.keras.losses.categorical_crossentropy(pred, pred)
            grads = tape.gradient(loss, model.trainable_variables)
            tf.keras.optimizers.legacy.Adam(learning_rate=params["cls_lr"]).apply_gradients(zip(grads, model.trainable_variables))
        else:
            e_t = model.get_model_output(
                latent,
                timestep,
                batch_size,
            )
            a_t, a_prev = alphas[index], alphas_prev[index]
            latent = model.get_x_prev(latent, e_t,  a_t, a_prev, params["temperature"])

    return latent

In [ ]:
'''
def run_gen_epoch():


    for i, (data, target) in tqdm(enumerate(agent.state["tr_loader"])):
        #if agent.state["sample_amt"] > agent.params["samples_per_task"] > 0: break
        if data.shape[0] != batch_size: break
        agent.state["sample_amt"] += data.shape[0]

        agent.state["data"] = data
        agent.state["target"] = target
        agent.state["i_example"] = i

        data = agent.state["data"]
        latent = agent.gen.encoder(data)
        mem_x = None

        for it in range(agent.params["gen_iters"]):
            generate(agent, input_latent=latent)
    if agent.state["epoch"] % agent.params["print_every"] == 0:

        print("\nEvaluate generator on Task: ", agent.state["task"], " Epoch: ", agent.state["epoch"])
        loss = []
        for i, (data, target) in tqdm(enumerate(agent.state["ts_loader"])):

            mem_x = generate(agent, input_latent=agent.encoder(data), train=False)
            mem_pred = agent.cls(mem_x)
            mem_loss = tf.keras.losses.categorical_crossentropy(mem_pred, mem_pred)
            loss.append(np.mean(mem_loss))

        print("Loss on generate: ",  np.mean(mem_loss))
'''

In [ ]:
model = UNetModel()

In [ ]:
params = {
    "num_steps": 100,
    "input_latent_strength":0.8,
    "temperature": 0.9,
    "batch_size": 256,
    "gen_lr": 2e-5,
}

In [ ]:
(X_train, y_train), (X_test, y_test) = load_cifar_10()

In [ ]:
for epoch in range(params["n_epoch"]):
    loss = []
    for i in range(0, X_train.shape[0], params["batch_size"]):
        X_batch = X_train[i:i+params["batch_size"]]
        y_batch = y_train[i:i+params["batch_size"]]
        latent = encoder(X_batch)
        mem_x = generate(input_latent=latent)
        mem_pred = model.cls(mem_x)
        mem_loss = tf.keras.losses.categorical_crossentropy(mem_pred, mem_pred)
        loss.append(np.mean(mem_loss))
    print("Loss on generate: ",  np.mean(loss))